**Q1)**
In this question a binary classification problem will be solved using sequential models on BBBP dataset.

In [1]:
student_id =  402300459
student_name = "Mohammadreza Bagheri"

print("your student id:", student_id)
print("your name:", student_name)

your student id: 402300459
your name: Mohammadreza Bagheri


Here, the required libraries and packages are imported.

In [27]:
import torch
import os
import pandas as pd
import numpy as np
!pip install rdkit
import torch.nn as nn
import torch.optim as optim
from rdkit import Chem
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold, train_test_split
from torchvision.transforms import Compose, ToTensor, Normalize, Lambda



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
print("your device is:", device.type)

your device is: cuda


In the following session, is specificed to connect to drive.

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from google.colab import drive
drive.mount('/content/drive')

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'DeepHW4/Q1'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Mounted at /content/drive
['BBBP.csv', 'Q1.ipynb', '__pycache__', 'Q1Classes.py', 'Q1(1).ipynb']


In [6]:
import sys
sys.path.append(GOOGLE_DRIVE_PATH)

import time, os
os.environ["TZ"] = "US/Eastern"
time.tzset()

Q1Classes_path = os.path.join(GOOGLE_DRIVE_PATH, 'Q1Classes.py')
Q1Classes_time = time.ctime(os.path.getmtime(Q1Classes_path))
print('Q1Classes.py last edited on %s' % Q1Classes_time)

Q1Classes.py last edited on Mon Jan  8 15:07:59 2024


In [7]:
import Q1Classes

Here, we load the bbbp dataset.

In [8]:
bbbp_dataset = pd.read_csv('/content/drive/MyDrive/DeepHW4/Q1/BBBP.csv')

**part 1)**
In this part, we Investigate some features and the type of values in BBBP dataset:

In [9]:
print("Dataset Shape:", bbbp_dataset.shape)
print("Columns:", bbbp_dataset.columns)
print("Data Types:", bbbp_dataset.dtypes)
print("class one percentile:", 100*sum(bbbp_dataset['p_np'])/bbbp_dataset.shape[0])
print("class zero percentile:", 100 - 100*sum(bbbp_dataset['p_np'])/bbbp_dataset.shape[0])

Dataset Shape: (2050, 4)
Columns: Index(['num', 'name', 'p_np', 'smiles'], dtype='object')
Data Types: num        int64
name      object
p_np       int64
smiles    object
dtype: object
class one percentile: 76.4390243902439
class zero percentile: 23.5609756097561


In [10]:
dataset_arr = np.array([len(s) for s in bbbp_dataset['smiles']])
min_length = min(dataset_arr)
max_length = max(dataset_arr)
mean_length = np.mean(dataset_arr)
var_length = np.var(dataset_arr)
print("Range of Molecule Length: {} to {}".format(min_length, max_length))
print("Mean Molecule Length:", mean_length)
print("Variance of Molecule Length:", var_length)

vocabulary = ['.', '(', ')', '+', '-', '/', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', '@', 'B', 'C', 'F', 'H', 'I','N', 'O', 'P', 'S', '[', ']', '\\', 'l', 'n', 'o', 'r', 's', 't']

Range of Molecule Length: 3 to 400
Mean Molecule Length: 51.47414634146342
Variance of Molecule Length: 937.1673803688282


**Part 2)**
In this part, we define a FC network to and train it on BBBP dataset to estimate the class of molecules.

In [11]:
from Q1Classes import BBBPDataset
dataset = BBBPDataset(bbbp_dataset['smiles'],bbbp_dataset['p_np'])

batch_size = 64

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(0.8 * len(dataset)), len(dataset)-int(0.8 * len(dataset))])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [70]:
from Q1Classes import Fully_connected_net
input_size = len(dataset[0][0])
hidden_size = 128
output_size = 1
model = Fully_connected_net(input_size, hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
model = Q1Classes.train_model(model, 10, train_loader, optimizer, criterion, device)



Epoch 9: 100%|██████████| 11/11 [00:00<00:00, 33.54batch/s, loss=0.305]


In [67]:
acc = Q1Classes.test_model_fc(model,test_loader)

Test Accuracy: 80.49%


**Part 3)**
In this part we solve the problem using LSTM network:

In [80]:
input_size = 14000   # Size of one-hot encoding for each input molecule
hidden_size = 128  # Hidden size of LSTM layer
num_classes = 2    # Number of output classes
epoch_num = 10
batch_size = 32
learning_rate = 0.001


model = Q1Classes.LSTMNet(input_size, hidden_size, num_classes).to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

model = Q1Classes.train_LSTM(model, epoch_num, train_loader, optimizer, criterion, device)

Epoch 9: 100%|██████████| 52/52 [00:01<00:00, 51.95batch/s, loss=0.000235]


In [15]:
Q1Classes.test_model_LSTM(model,test_loader)

Test Accuracy: 84.63%


**part 4)** In this part, we use BiLSTM structure, instead of working with traditional LSTM:

In [16]:
input_size = 14000   # Size of one-hot encoding for each input molecule
hidden_size = 128  # Hidden size of LSTM layer
num_classes = 2    # Number of output classes
num_epochs = 10
batch_size = 32
learning_rate = 0.001
num_layers = 2

model = Q1Classes.BiLSTMClassifier(input_size, hidden_size, num_layers, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [17]:
model = Q1Classes.train_LSTM(model, epoch_num, train_loader, optimizer, criterion, device)

Epoch 9: 100%|██████████| 26/26 [00:01<00:00, 24.38batch/s, loss=0.00837]


In [18]:
Q1Classes.test_model_LSTM(model,test_loader)

Test Accuracy: 85.12%


**Part 5)** In this part, we train the FC model and LSTM model using k-fold cross-validation technique.

In [23]:
k_folds = 5
input_size = len(dataset[0][0])
hidden_size = 128
output_size = 1

kfold = KFold(n_splits=k_folds, shuffle=True)

for fold, (train_indices, val_indices) in enumerate(kfold.split(dataset)):


    # Split data into training and validation sets
    train_data = torch.stack([dataset[i][0] for i in train_indices])
    train_labels = [dataset[i][1] for i in train_indices]
    train_data = train_data.to(device)
    train_labels = torch.tensor(train_labels).to(device)

    val_data = torch.stack([dataset[i][0] for i in val_indices])
    val_labels = [dataset[i][1] for i in val_indices]
    val_data = val_data.to(device)
    val_labels = torch.tensor(val_labels).to(device)


    model = Q1Classes.Fully_connected_net(input_size, hidden_size, output_size).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


    for epoch in range(num_epochs):
        model.train()
        outputs = model(train_data.permute(0,2,1))
        loss = criterion(outputs.squeeze(), train_labels.float().squeeze())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    model.eval()
    val_outputs = model(val_data.permute(0,2,1))
    val_loss = criterion(val_outputs.squeeze(), val_labels.float().squeeze())
    _, val_predictions = torch.max(val_outputs, 1)
    total = val_labels.size(0)
    correct = (val_predictions.squeeze() == val_labels.squeeze()).sum().item()
    print(f'Fold {fold + 1}/{k_folds}, Validation Loss: {val_loss.item()}, Validation Accuracy: {(correct)/(total) * 100}%')

Fold 1/5, Validation Loss: 1875.1451416015625, Validation Accuracy: 23.414634146341466%
Fold 2/5, Validation Loss: 1814.0897216796875, Validation Accuracy: 25.853658536585368%
Fold 3/5, Validation Loss: 1899.208740234375, Validation Accuracy: 22.439024390243905%
Fold 4/5, Validation Loss: 1893.853759765625, Validation Accuracy: 22.682926829268293%
Fold 5/5, Validation Loss: 1873.787841796875, Validation Accuracy: 23.414634146341466%


In [25]:
kfold = KFold(n_splits=k_folds, shuffle=True)

for fold, (train_indices, val_indices) in enumerate(kfold.split(dataset)):



    train_data = torch.stack([dataset[i][0] for i in train_indices])
    train_labels = [dataset[i][1] for i in train_indices]
    train_data = train_data.to(device)
    train_labels = torch.tensor(train_labels).to(device)

    val_data = torch.stack([dataset[i][0] for i in val_indices])
    val_labels = [dataset[i][1] for i in val_indices]
    val_data = val_data.to(device)
    val_labels = torch.tensor(val_labels).to(device)


    model = Q1Classes.BiLSTMClassifier(input_size, hidden_size, num_layers, num_classes).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


    for epoch in range(num_epochs):
        model.train()
        outputs = model(train_data.permute(0,2,1))
        loss = criterion(outputs, train_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    val_outputs = model(val_data.permute(0,2,1))
    val_loss = criterion(val_outputs, val_labels)
    _, val_predictions = torch.max(val_outputs, 1)

    print(f'Fold {fold + 1}/{k_folds}, Validation Loss: {val_loss.item()},  Validation Accuracy: {(val_predictions == val_labels).sum().item() / val_labels.size(0) * 100}%')

Fold 1/5, Validation Loss: 0.4902384281158447,  Validation Accuracy: 78.04878048780488%
Fold 2/5, Validation Loss: 0.48935574293136597,  Validation Accuracy: 77.07317073170732%
Fold 3/5, Validation Loss: 0.5108120441436768,  Validation Accuracy: 77.07317073170732%
Fold 4/5, Validation Loss: 0.5149846076965332,  Validation Accuracy: 75.85365853658537%
Fold 5/5, Validation Loss: 0.4771016836166382,  Validation Accuracy: 79.02439024390245%


**Part 6)**

In traditional RNN, decisions are made only based on the previous observations. In bidirectional RNN, however, decisions are made based on both previous and future observations. This might be the main reason for the difference between the quality of these networks.

In [58]:
b = bbbp_dataset['smiles'].str.len()
a = b.sort_values().index

smiles = []
labels = []
for i in range(len(bbbp_dataset)):
    smiles.append(bbbp_dataset['smiles'][a[i]])
    labels.append(bbbp_dataset['p_np'][a[i]])

In [90]:
from Q1Classes import Fully_connected_net



num_bins = 10
length = len(bbbp_dataset)
dataset = BBBPDataset(bbbp_dataset['smiles'],bbbp_dataset['p_np'])

batch_size = 16

acc_fc = []


for i in range(num_bins):
    dataset_i = BBBPDataset(smiles[i*(length//10)+1:(i+1)*(length//10)],labels[i*(length//10)+1:(i+1)*(length//10)])

    train_dataset, test_dataset = torch.utils.data.random_split(dataset_i, [int(0.8 * len(dataset_i)), len(dataset_i)-int(0.8 * len(dataset_i))])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


    input_size = len(dataset[0][0])
    hidden_size = 128
    output_size = 1
    model = Fully_connected_net(input_size, hidden_size, output_size)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model = Q1Classes.train_model(model, 10, train_loader, optimizer, criterion, device)
    acc_fc.append(Q1Classes.test_model_fc(model,test_loader))






Epoch 9: 100%|██████████| 11/11 [00:00<00:00, 31.08batch/s, loss=0.208]


Test Accuracy: 87.80%


Epoch 9: 100%|██████████| 11/11 [00:00<00:00, 32.87batch/s, loss=0.0647]


Test Accuracy: 82.93%


Epoch 9: 100%|██████████| 11/11 [00:00<00:00, 23.65batch/s, loss=0.00478]


...

Test Accuracy: 75.61%


Epoch 9: 100%|██████████| 11/11 [00:00<00:00, 28.59batch/s, loss=0.00404]


Test Accuracy: 90.24%


Epoch 9: 100%|██████████| 11/11 [00:00<00:00, 23.73batch/s, loss=0.0768]


Test Accuracy: 80.49%


In [87]:
acc_lstm = []

for i in range(num_bins):
    dataset_i = BBBPDataset(smiles,labels)

    train_dataset, test_dataset = torch.utils.data.random_split(dataset_i, [int(0.8 * len(dataset_i)), len(dataset_i)-int(0.8 * len(dataset_i))])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


    input_size = 14000   # Size of one-hot encoding for each input molecule
    hidden_size = 128  # Hidden size of LSTM layer
    num_classes = 2    # Number of output classes
    epoch_num = 10
    batch_size = 32
    learning_rate = 0.001


    model = Q1Classes.LSTMNet(input_size, hidden_size, num_classes).to(device)


    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    model = Q1Classes.train_LSTM(model, epoch_num, train_loader, optimizer, criterion, device)
    acc_lstm.append(Q1Classes.test_model_LSTM(model,test_loader))



Epoch 9: 100%|██████████| 103/103 [00:01<00:00, 66.66batch/s, loss=4.69e-6]


Test Accuracy: 88.29%


Epoch 9: 100%|██████████| 52/52 [00:01<00:00, 49.06batch/s, loss=1.79e-5]


Test Accuracy: 85.61%


Epoch 9: 100%|██████████| 52/52 [00:01<00:00, 38.13batch/s, loss=0.0115]


...

Test Accuracy: 86.34%


Epoch 9: 100%|██████████| 52/52 [00:01<00:00, 50.93batch/s, loss=0.00295]


Test Accuracy: 84.88%


Epoch 9: 100%|██████████| 52/52 [00:01<00:00, 50.13batch/s, loss=2.6e-5]


Test Accuracy: 89.76%


In [88]:
acc_bi = []
for i in range(num_bins):
    dataset_i = BBBPDataset(smiles[i*(length//10)+1:(i+1)*(length//10)],labels[i*(length//10)+1:(i+1)*(length//10)])

    train_dataset, test_dataset = torch.utils.data.random_split(dataset_i, [int(0.8 * len(dataset_i)), len(dataset_i)-int(0.8 * len(dataset_i))])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



    input_size = 14000   # Size of one-hot encoding for each input molecule
    hidden_size = 128  # Hidden size of LSTM layer
    num_classes = 2    # Number of output classes
    num_epochs = 10
    batch_size = 32
    learning_rate = 0.001
    num_layers = 2
    model_bi = Q1Classes.BiLSTMClassifier(input_size, hidden_size, num_layers, num_classes).to(device)
    optimizer = torch.optim.Adam(model_bi.parameters(), lr=learning_rate)
    model_bi = Q1Classes.train_LSTM(model_bi, epoch_num, train_loader, optimizer, criterion, device)
    acc_bi.append(Q1Classes.test_model_LSTM(model_bi,test_loader))


Epoch 9: 100%|██████████| 6/6 [00:00<00:00, 61.98batch/s, loss=0.355]


Test Accuracy: 87.80%


Epoch 9: 100%|██████████| 6/6 [00:00<00:00, 56.19batch/s, loss=0.118]


Test Accuracy: 90.24%


Epoch 9: 100%|██████████| 6/6 [00:00<00:00, 52.81batch/s, loss=0.0037]


...

Test Accuracy: 82.93%


Epoch 9: 100%|██████████| 6/6 [00:00<00:00, 39.35batch/s, loss=0.000221]


Test Accuracy: 82.93%


Epoch 9: 100%|██████████| 6/6 [00:00<00:00, 31.17batch/s, loss=8.18e-5]


Test Accuracy: 87.80%


In [93]:
print(acc_fc)
print(acc_lstm)
print(acc_bi)

[87.8048780487805, 82.92682926829268, 85.36585365853658, 95.1219512195122, 90.2439024390244, 92.6829268292683, 78.04878048780488, 75.60975609756098, 90.2439024390244, 80.48780487804878]
[88.29268292682927, 85.60975609756098, 88.53658536585365, 85.85365853658537, 86.34146341463415, 84.63414634146342, 86.58536585365853, 86.34146341463415, 84.8780487804878, 89.7560975609756]
[87.8048780487805, 90.2439024390244, 90.2439024390244, 90.2439024390244, 90.2439024390244, 70.73170731707317, 78.04878048780488, 82.92682926829268, 82.92682926829268, 87.8048780487805]


We expect the test accuracy of LSTM models to increase by increasing the length of smile codes. Its because these networks include a memory cell sufficient to hold long term patterns. However, the change in the length of smile codes might not be effective on the accuracy of FC or traditional recurrent networks, because they could not hold memory in long term.